# **Reciprocal space and Brillouin zone**

<i class="fa fa-home fa-2x"></i><a href="../index.ipynb" style="font-size: 20px"> Go back to index</a>

**Source code:** https://github.com/osscar-org/quantum-mechanics/blob/master/notebook/band-theory/brillouin_zone.ipynb
<p style="text-align: justify;font-size:15px">
    In this notebook we explore two central concepts in electronic structure and band theory: those of reciprocal space and the Brillouin zone.
</p>
<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## **Goals**
* Appreciate the nature of the relationship between the direct and reciprocal space descriptions of a crystalline system.
* Use this understanding to explain the differences in the Brillouin zones observed for different crystal structures. 


<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

In [57]:
from widget_bzvisualizer import BZVisualizer
import numpy as np
import seekpath
import nglview as nv
from ase.build import bulk, molecule
from ipywidgets import HBox, VBox, Button, Output, Text, Tab, Layout, Label, HTML, Dropdown
import ipywidgets as widgets
import nglview as nv
from ase import Atom, Atoms
from ase.symbols import symbols2numbers
from ase.data import atomic_numbers

In [61]:
bt_compute = Button(description="Update")

mat = bulk('Cu', 'fcc', a=3.6)
mat = mat.repeat([3, 3, 3])


m = nv.NGLWidget(width='400px', height='400px')
m.background='black'
c1 = m.add_component(nv.ASEStructure(mat))
m.clear()
# m.add_ball_and_stick()
m.add_unitcell()

mpos = [];
msym = [];

for i in mat:
    mpos.append(i.position.tolist())
    msym.append('Cu')


In [ ]:
w = BZVisualizer(mat.cell.tolist(), mpos, symbols2numbers(mat.get_chemical_symbols()), 
                 True, height='400px', width='400px')


"""
now contents are dropdwn menus. 
Depending on the crystal family selected, there shall be a dropdown menu 

"""
crystal_family_dropdown = widgets.Dropdown(options=['Cubic','Orthorombic','Tetragonal','Triclinic','Monoclinic','Hexagonal','Rhombohedral'],
                                           value='Cubic',
                                           description='Crystal family:',
                                           disabled=False)

centering_dropdown = widgets.Dropdown(options=['Primitive','Base-centered','Body-centered', 'Face-centered'],
                                           value='Primitive',
                                           description='Crystal family:',
                                           disabled=False)

tab_contents = ['Crystal family', 'Centering'] 
# children = [widgets.Text(description=name) for name in tab_contents]
children = [crystal_family_dropdown, centering_dropdown]

tab = widgets.Tab()
tab.children = children

# tab.set_title(0, 'Crystal family')
# tab.set_title(1, 'b')
# tab.set_title(2, 'c')

tab.set_title(0, 'Crystal family')
tab.set_title(1, 'Centering')

# tab.children[0].value = str(mat.cell[0].tolist())
# tab.children[1].value = str(mat.cell[1].tolist())
# tab.children[2].value = str(mat.cell[2].tolist())

bt_add = Button(description = "Add")
bt_clear = Button(description = "Clear all")

atom_pos = Text(description="Atom position")
atom_num = Text(description="Atom symbol", layout=Layout(width='120px'))

output = Output(layout=Layout(width="400px"))

ed_index = Text(description="Index", layout=Layout(width='120px'))
ed_sym = Text(description="Atom symbol", layout=Layout(width='120px'))
ed_pos = Text(description="Atom position")
ed_get = Button(description="Get")
ed_set = Button(description="Set")

def update_elements(c):
    with output:
        output.clear_output()
        for i,j in enumerate(mpos):
            print(i, msym[i], j)

update_elements('init')

def add_atom(c):
    global mpos
    
    a = list(eval(atom_pos.value))
    b = atom_num.value
    
    assert type(a) == list
    assert type(b) == str
    
    mpos.append(a)
    msym.append(b)
    atom_pos.value = ''
    atom_num.value = ''
    mat.append(Atom(b, a))
    update_elements('add')
    
bt_add.on_click(add_atom)

def clear_atom(c):
    global mpos, msym
    mpos= [];
    msym = [];
    
    for i in range(len(mat)):
        mat.pop()
        
    update_elements('clear')
    
bt_clear.on_click(clear_atom)

def get_atom(c):
    ed_sym.value = mat.get_chemical_symbols()[int(ed_index.value)]
    ed_pos.value = str(mat[int(ed_index.value)].position.tolist())
    
ed_get.on_click(get_atom)

def set_atom(c):
    mat[int(ed_index.value)].symbol = ed_sym.value
    mat[int(ed_index.value)].position = np.array(eval(ed_pos.value))
    
    msym[int(ed_index.value)] = ed_sym.value
    mpos[int(ed_index.value)] = np.array(eval(ed_pos.value))
    
    ed_sym.value = ''
    ed_pos.value = ''
    
    update_elements('set')
    
ed_set.on_click(set_atom)
    
tab2 = Tab()

a_slider=widgets.FloatSlider(
    value=3.5,
    min=0,
    max=10.0,
    step=0.1,
    description='a:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

b_slider=widgets.FloatSlider(
    value=3.5,
    min=0,
    max=10.0,
    step=0.1,
    description='b:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

c_slider=widgets.FloatSlider(
    value=3.5,
    min=0,
    max=10.0,
    step=0.1,
    description='c:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

alpha_slider=widgets.FloatSlider(
    value=90.,
    min=0,
    max=180,
    step=0.1,
    description=r'$\alpha$:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

beta_slider=widgets.FloatSlider(
    value=90.,
    min=0,
    max=180,
    step=0.1,
    description=r'$\beta$:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

gamma_slider=widgets.FloatSlider(
    value=90.,
    min=0,
    max=180,
    step=0.1,
    description=r'$\gamma$:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)


def fix_sliders_ab(a):
    # fix value of b to that of a
    return a

def fix_sliders_ac(a):
    # fix value of b and c sliders to that of a
    return a

def fix_sliders_alpha_beta(alpha):
    # fix value of b and c sliders to that of a
    return alpha

def fix_sliders_alpha_gamma(alpha):
    # fix value of b and c sliders to that of a
    return alpha

# set up link between a and b sliders
l_ab = widgets.dlink((a_slider, 'value'), (b_slider, 'value'),transform=fix_sliders_ab)

# set up link between a and c sliders
l_ac = widgets.dlink((a_slider, 'value'), (c_slider, 'value'),transform=fix_sliders_ac)

# set up link between alpha and beta sliders
l_alpha_beta = widgets.dlink((alpha_slider, 'value'), (beta_slider, 'value'),transform=fix_sliders_alpha_beta)
l_alpha_beta.unlink()

# set up link between alpha and gamma sliders
l_alpha_gamma = widgets.dlink((alpha_slider, 'value'), (gamma_slider, 'value'),transform=fix_sliders_alpha_gamma)
l_alpha_gamma.unlink()

# default settings for cubic crystal type
alpha_slider.value=90
beta_slider.value=90
gamma_slider.value=90
alpha_slider.disabled=True
beta_slider.disabled=True
gamma_slider.disabled=True
l_ab.link()
b_slider.disabled=True
l_ac.link()
c_slider.disabled=True


tab2.children = [VBox([a_slider,b_slider,c_slider]), VBox([alpha_slider,beta_slider,gamma_slider])]

# tab2.children = [VBox([HBox([atom_num, atom_pos]), HBox([bt_add, bt_clear])]), 
#                  VBox([HBox([ed_sym, ed_pos]), HBox([ed_index, ed_get, ed_set])]), output]

tab2.set_title(0, 'Lattice vectors')
tab2.set_title(1, 'Axial angles')
tab2.set_title(2, 'List of atoms')

debug_output =widgets.Output(layout={'border': '1px solid black'})

# display(debug_output)

# Function to set up lattice vector and axial angle selection according to choice 
# of crystal family + centering
def reset_sliders(change):
    # crystal_family = str(tab.children[0].value)
    global crystal_family_dropdown
    if change['type'] == 'change' and change['name'] == 'value':

        crystal_family = str(change.new)

       

        if(crystal_family=='Cubic'):
            with debug_output:
                print("crystal_family=="+str(crystal_family))  
            # lock b and c sliders to that of a
            # lock angles to 90,90,90
          
            alpha_slider.value=90
            beta_slider.value=90
            gamma_slider.value=90
            alpha_slider.disabled=True
            beta_slider.disabled=True
            gamma_slider.disabled=True
            l_ab.link()
            b_slider.disabled=True
            l_ac.link()
            c_slider.disabled=True
            l_alpha_beta.unlink()
            l_alpha_gamma.unlink()

        elif(crystal_family=='Orthorombic'):
            with debug_output:
                print("crystal_family=="+str(crystal_family))  
            # vector lengths are all independent
            # lock angles to 90,90,90
           
            alpha_slider.value=90
            beta_slider.value=90
            gamma_slider.value=90
            alpha_slider.disabled=True
            beta_slider.disabled=True
            gamma_slider.disabled=True
            l_ab.unlink()
            l_ac.unlink()
            b_slider.disabled=False
            c_slider.disabled=False
            l_alpha_beta.unlink()
            l_alpha_gamma.unlink()


        elif(crystal_family=='Tetragonal'):
            with debug_output:
                print("crystal_family=="+str(crystal_family))

            # lock b slider value to that of a
            alpha_slider.value=90
            beta_slider.value=90
            gamma_slider.value=90
            alpha_slider.disabled=True
            beta_slider.disabled=True
            gamma_slider.disabled=True
            l_ab.link()
            b_slider.disabled=True
            l_ac.unlink()
            c_slider.disabled=False
            l_alpha_beta.unlink()
            l_alpha_gamma.unlink()

        elif(crystal_family=='Monoclinic'):
            # two lattice vectors meet at 90 degrees.
            # The third meets at an angle beta
            # all 3 lattice vectors can take unique values
            
            
            alpha_slider.value=90
            gamma_slider.value=90
            alpha_slider.disabled=True
            gamma_slider.disabled=True
            l_ab.unlink()
            l_ac.unlink()
            b_slider.disabled=False
            c_slider.disabled=False
            l_alpha_beta.unlink()
            l_alpha_gamma.unlink()
            
        elif(crystal_family=='Hexagonal'):
            # a=b with 120 degrees between
            # a and c are perpendicular. Similarly for b and c
            # all 3 lattice vectors can take unique values
          
            alpha_slider.value=90
            beta_slider.value=90
            gamma_slider.value=120
            alpha_slider.disabled=True
            gamma_slider.disabled=True
            l_ab.link()
            l_ac.unlink()
            b_slider.disabled=True
            c_slider.disabled=False
            l_alpha_beta.unlink()
            l_alpha_gamma.unlink()

        elif(crystal_family=='Rhombohedral'):
            # a=b with 120 degrees between
            # a and c are perpendicular. Similarly for b and c
            # all 3 lattice vectors can take unique values

            beta_slider.disabled=True
            gamma_slider.disabled=True
            l_ab.link()
            l_ac.link()
            b_slider.disabled=True
            c_slider.disabled=True
            l_alpha_beta.link()
            l_alpha_gamma.link()
            
        elif(crystal_family=='Triclinic'):
            # all 3 lattice vectors and angles may vary independently
            alpha_slider.disabled=False
            beta_slider.disabled=False
            gamma_slider.disabled=False
            l_ab.unlink()
            l_ac.unlink()
            b_slider.disabled=False
            c_slider.disabled=False
            l_alpha_beta.unlink()
            l_alpha_gamma.unlink()
        
            
crystal_family_dropdown.observe(reset_sliders)


def compute_BZ(c):
    global c1,mat
    # get mag of lattice vec from slider and construct vector
    a = a_slider.value
    b = b_slider.value
    c = c_slider.value
    # with debug_output:
    #     print("a="+str(a))
    v1=[a,0.,0.]
    v2=[0.,b,0.]
    v3=[0.,0.,c]

#     v1 = list(eval(tab.children[0].value))
#     v2 = list(eval(tab.children[1].value))
#     v3 = list(eval(tab.children[2].value))
    
#     assert type(v1) == list
#     assert type(v2) == list
#     assert type(v3) == list
        
        
        
    mat.set_cell([v1, v2, v3])
    m.remove_component(c1)
    c1 = m.add_component(nv.ASEStructure(mat))
    m.clear()
    # m.add_ball_and_stick()
    m.add_unitcell()
    w.numbers = symbols2numbers(mat.get_chemical_symbols())
    w.positions = mpos
    w.cell = mat.cell.tolist()
    w.update_structure += 1


bt_compute.on_click(compute_BZ);

label1 = HTML(value = f"<div style='width: 400px; text-align:center;'><b><font color='blue'><font size=5>Structure</b></div>")
label2 = HTML(value = f"<div style='width: 400px; text-align:center;'><b><font color='blue'><font size=5>Brillouin zone</b></div>")

display(HBox([label1, label2]), HBox([m, w]), HBox([tab, tab2]), bt_compute)